In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
from jax.lib import xla_bridge
print("jax backend {}".format(xla_bridge.get_backend().platform))

from SSN_classes import SSN_mid_local
from training import new_two_stage_training


jax backend cpu


In [2]:
import jax.numpy as np

from parameters import grid_pars, filter_pars, stimuli_pars, sig_pars, ssn_pars, conn_pars_m, conn_pars_s, ssn_layer_pars, conv_pars, training_pars, loss_pars

'''# Calculate A by using the default find_A function of SSN_mid_local when A is undefined
ssn_mid = SSN_mid_local(
    ssn_pars=ssn_pars, # got c_E and c_I as new parameters
    grid_pars=grid_pars,
    conn_pars=conn_pars_m,
    filter_pars=filter_pars,
    J_2x2 = ssn_layer_pars.J_2x2_m,
    gE = ssn_layer_pars.gE_m, 
    gI = ssn_layer_pars.gE_m
)

ssn_pars.A = ssn_mid.A
if ssn_pars.phases == 4:
    ssn_pars.A2 = ssn_mid.A2
'''
# check where A2 is defined in Clara's code
ssn_ori_map_loaded = np.load(os.path.join(os.getcwd(), 'ssn_map_uniform_good.npy'))
#Find normalization constant of Gabor filters
ssn_mid=SSN_mid_local(ssn_pars=ssn_pars, grid_pars=grid_pars, conn_pars=conn_pars_m, filter_pars=filter_pars, J_2x2= ssn_layer_pars.J_2x2_m, gE = ssn_layer_pars.gE_m, gI=ssn_layer_pars.gI_m, ori_map = ssn_ori_map_loaded)
ssn_pars.A = ssn_mid.A
ssn_pars.A2 = ssn_mid.A2
####################### TRAINING PARAMETERS #############################
constant_ssn_pars = dict(
    ssn_pars=ssn_pars,
    grid_pars=grid_pars,
    conn_pars_m=conn_pars_m,
    conn_pars_s=conn_pars_s,
    gE=ssn_layer_pars.gE,
    gI=ssn_layer_pars.gI,
    filter_pars=filter_pars,
    conv_pars=conv_pars,# this got a lot of new stuff in it that might be a problem
    loss_pars=loss_pars,
    noise_type="poisson",
)
########### TRAINING ############
([ssn_layer_pars_dict, readout_pars],
    val_loss_per_epoch,
    training_losses,
    training_accs,
    train_sig_inputs,
    train_sig_outputs,
    val_sig_inputs,
    val_sig_outputs,
    epoch_c,
    save_w_sigs) = new_two_stage_training(
    ssn_layer_pars,
    sig_pars,
    training_pars,
    constant_ssn_pars,
    stimuli_pars,
    extra_stop=2,
    ssn_ori_map=ssn_ori_map_loaded
    )

Loading orientation map
#### NOT SAVING! ####
Training loss: 1.6277848482131958 ¦ Validation -- loss: 1.4607200622558594, true accuracy: 0.5, at epoch 0, (time 20.67272114753723, 0.3209366798400879), 
Training accuracy: 0.46000000834465027, all losses[[1.196635  ]
 [0.09357131]
 [0.27119902]
 [0.06637964]
 [0.        ]
 [1.6277848 ]]
Training loss: 1.559080719947815 ¦ Validation -- loss: 1.601881980895996, true accuracy: 0.41999998688697815, at epoch 1, (time 19.234965562820435, 0.3137853145599365), 
Training loss: 1.523915410041809 ¦ Validation -- loss: 1.368055820465088, true accuracy: 0.5199999809265137, at epoch 3, (time 0.8002970218658447, 0.30969738960266113), 
Training loss: 1.1943503618240356 ¦ Validation -- loss: 1.4193984270095825, true accuracy: 0.46000000834465027, at epoch 5, (time 0.7794954776763916, 0.3204319477081299), 
Entering second stage at epoch 5
Training loss: 1.4644416570663452 ¦ Validation -- loss: 1.2910908460617065, true accuracy: 0.5799999833106995, at epoch

In [3]:
from training_supp import sep_exponentiate
print(sep_exponentiate(ssn_layer_pars_dict['logJ_2x2'][0]))
print(ssn_layer_pars.J_2x2_m)
print(sep_exponentiate(ssn_layer_pars_dict['logJ_2x2'][1]))
print(ssn_layer_pars.J_2x2_s)

[[ 1.9254755 -1.0111731]
 [ 3.65578   -1.6944369]]
[[ 1.935     -1.0062   ]
 [ 3.6377997 -1.7028   ]]
[[ 4.4198217 -1.6662824]
 [ 5.0534315 -1.2356309]]
[[ 4.4413204 -1.6582118]
 [ 5.0289063 -1.2416493]]
